In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [3]:
df=pd.read_csv('new_labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,./images\N1.xml,1093,1396,645,727
1,./images\N100.xml,134,301,312,350
2,./images\N101.xml,31,139,128,161
3,./images\N102.xml,164,316,216,243
4,./images\N103.xml,813,1067,665,724


In [4]:
import xml.etree.ElementTree as xet

In [5]:
filename= df['filepath'][0]
filename

'./images\\N1.xml'

In [6]:
def getfilename(filename):
    filename_image=xet.parse(filename).getroot().find('filename').text
    filepath_image=os.path.join('./images',filename_image)
    return filepath_image
#we use filepath here beacuse in opencv we need filepath instead of filename
    

In [7]:
getfilename(filename)

'./images\\N1.jpeg'

In [8]:
image_path= list(df['filepath'].apply(getfilename))
image_path

['./images\\N1.jpeg',
 './images\\N100.jpeg',
 './images\\N101.jpeg',
 './images\\N102.jpeg',
 './images\\N103.jpeg',
 './images\\N104.jpeg',
 './images\\N105.jpeg',
 './images\\N106.jpeg',
 './images\\N107.jpeg',
 './images\\N108.jpeg',
 './images\\N109.jpeg',
 './images\\N11.jpeg',
 './images\\N110.jpeg',
 './images\\N111.jpeg',
 './images\\N112.jpeg',
 './images\\N113.jpeg',
 './images\\N114.jpeg',
 './images\\N116.jpeg',
 './images\\N117.jpeg',
 './images\\N118.jpeg',
 './images\\N119.jpeg',
 './images\\N12.jpeg',
 './images\\N120.jpeg',
 './images\\N121.jpeg',
 './images\\N122.jpeg',
 './images\\N123.jpeg',
 './images\\N124.jpeg',
 './images\\N126.jpeg',
 './images\\N127.jpeg',
 './images\\N128.jpeg',
 './images\\N129.jpeg',
 './images\\N130.jpeg',
 './images\\N131.jpeg',
 './images\\N132.jpeg',
 './images\\N133.jpeg',
 './images\\N134.jpeg',
 './images\\N135.jpeg',
 './images\\N136.jpeg',
 './images\\N137.jpeg',
 './images\\N138.jpeg',
 './images\\N139.jpeg',
 './images\\N14.jpeg

# verify image and output

In [9]:
filepath= image_path[0]
filepath


'./images\\N1.jpeg'

In [10]:
img=cv2.imread(filepath)
#window to show file
cv2.namedWindow('example',cv2.WINDOW_NORMAL) #with this we can fit image into window frame
#show image
cv2.imshow('example',img)
#termination of windows
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
#bounding box creation using xmin, xmax,ymin,ymax
#1093 	1396 	645 	727
cv2.rectangle(img, (1093,645),(1396,727),(255,0,255),3)#pixel=thickness of 3
cv2.namedWindow('example',cv2.WINDOW_NORMAL) #with this we can fit image into window frame
#show image
cv2.imshow('example',img)
#termination of windows
cv2.waitKey(0)
cv2.destroyAllWindows()

# data preprocessing

In [12]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [13]:
labels=df.iloc[:,1:].values

In [14]:
data=[]
output=[]
for ind in range(len(image_path)):
    image=image_path[ind]
    img_arr= cv2.imread(image)
    h,w,d=img_arr.shape
    #preprocessing
    load_image=load_img(image,target_size=(224,224))
    load_image_arr=img_to_array(load_image)
    #normalisation of input
    norm_load_img_arr= load_image_arr/255.0
    xmin,xmax,ymin,ymax=labels[ind]
    nxmin,nxmax=xmin/w, xmax/w
    nymin,nymax= ymin/h,ymax/h
    labels_norm= (nxmin,nxmax,nymax,nymin) #normalised output
    data.append(norm_load_img_arr)
    output.append(labels_norm)
    
    

# split data into train and test

In [15]:
x=np.array(data,dtype=np.float32)
y=np.array(output,dtype=np.float32)

In [16]:
x.shape,y.shape #in x total 225 image and of 224x224x3 and in y total 225 image with 4 outputs as xmin,xmax,ymin,ymax

((225, 224, 224, 3), (225, 4))

In [70]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((180, 224, 224, 3), (45, 224, 224, 3), (180, 4), (45, 4))

# deep learning model

In [33]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense,Flatten,Dropout,Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [84]:
inception_resnet= InceptionResNetV2(weights='imagenet',include_top=False,input_tensor=Input(shape=(224,224,3)))
inception_resnet.trainable=False
head_model= inception_resnet.output
head_model= Flatten()(head_model)
head_model=Dense(500,activation='relu')(head_model)
head_model=Dense(250,activation='relu')(head_model)
head_model=Dense(250,activation='relu')(head_model)
head_model=Dense(125,activation='relu')(head_model)
head_model=Dense(4,activation='sigmoid')(head_model)

model=Model(inputs=inception_resnet.input,outputs=head_model)

In [85]:
#model compile
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),metrics = ["accuracy"])
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_812 (Conv2D)         (None, 111, 111, 32)         864       ['input_5[0][0]']             
                                                                                                  
 batch_normalization_812 (B  (None, 111, 111, 32)         96        ['conv2d_812[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_812 (Activation  (None, 111, 111, 32)         0         ['batch_normalization_81

                                                                                                  
 conv2d_823 (Conv2D)         (None, 25, 25, 64)           12288     ['average_pooling2d_4[0][0]'] 
                                                                                                  
 batch_normalization_817 (B  (None, 25, 25, 96)           288       ['conv2d_817[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 batch_normalization_819 (B  (None, 25, 25, 64)           192       ['conv2d_819[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 batch_normalization_822 (B  (None, 25, 25, 96)           288       ['conv2d_822[0][0]']          
 atchNorma

                                                                                                  
 block35_1_conv (Conv2D)     (None, 25, 25, 320)          41280     ['block35_1_mixed[0][0]']     
                                                                                                  
 custom_scale_layer_160 (Cu  (None, 25, 25, 320)          0         ['mixed_5b[0][0]',            
 stomScaleLayer)                                                     'block35_1_conv[0][0]']      
                                                                                                  
 block35_1_ac (Activation)   (None, 25, 25, 320)          0         ['custom_scale_layer_160[0][0]
                                                                    ']                            
                                                                                                  
 conv2d_833 (Conv2D)         (None, 25, 25, 32)           10240     ['block35_1_ac[0][0]']        
          

                                                                                                  
 batch_normalization_840 (B  (None, 25, 25, 48)           144       ['conv2d_840[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_837 (Activation  (None, 25, 25, 32)           0         ['batch_normalization_837[0][0
 )                                                                  ]']                           
                                                                                                  
 activation_840 (Activation  (None, 25, 25, 48)           0         ['batch_normalization_840[0][0
 )                                                                  ]']                           
                                                                                                  
 conv2d_83

 atchNormalization)                                                                               
                                                                                                  
 activation_842 (Activation  (None, 25, 25, 32)           0         ['batch_normalization_842[0][0
 )                                                                  ]']                           
                                                                                                  
 activation_844 (Activation  (None, 25, 25, 32)           0         ['batch_normalization_844[0][0
 )                                                                  ]']                           
                                                                                                  
 activation_847 (Activation  (None, 25, 25, 64)           0         ['batch_normalization_847[0][0
 )                                                                  ]']                           
          

...

                                                                                                  
 batch_normalization_982 (B  (None, 5, 5, 256)            768       ['conv2d_982[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_979 (Activation  (None, 5, 5, 192)            0         ['batch_normalization_979[0][0
 )                                                                  ]']                           
                                                                                                  
 activation_982 (Activation  (None, 5, 5, 256)            0         ['batch_normalization_982[0][0
 )                                                                  ]']                           
                                                                                                  
 block8_2_

                                                                                                  
 batch_normalization_987 (B  (None, 5, 5, 192)            576       ['conv2d_987[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 batch_normalization_990 (B  (None, 5, 5, 256)            768       ['conv2d_990[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_987 (Activation  (None, 5, 5, 192)            0         ['batch_normalization_987[0][0
 )                                                                  ]']                           
                                                                                                  
 activatio

 conv2d_995 (Conv2D)         (None, 5, 5, 192)            399360    ['block8_5_ac[0][0]']         
                                                                                                  
 conv2d_998 (Conv2D)         (None, 5, 5, 256)            172032    ['activation_997[0][0]']      
                                                                                                  
 batch_normalization_995 (B  (None, 5, 5, 192)            576       ['conv2d_995[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 batch_normalization_998 (B  (None, 5, 5, 256)            768       ['conv2d_998[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activatio

 activation_1005 (Activatio  (None, 5, 5, 224)            0         ['batch_normalization_1005[0][
 n)                                                                 0]']                          
                                                                                                  
 conv2d_1003 (Conv2D)        (None, 5, 5, 192)            399360    ['block8_7_ac[0][0]']         
                                                                                                  
 conv2d_1006 (Conv2D)        (None, 5, 5, 256)            172032    ['activation_1005[0][0]']     
                                                                                                  
 batch_normalization_1003 (  (None, 5, 5, 192)            576       ['conv2d_1003[0][0]']         
 BatchNormalization)                                                                              
                                                                                                  
 batch_nor

 batch_normalization_1013 (  (None, 5, 5, 224)            672       ['conv2d_1013[0][0]']         
 BatchNormalization)                                                                              
                                                                                                  
 activation_1013 (Activatio  (None, 5, 5, 224)            0         ['batch_normalization_1013[0][
 n)                                                                 0]']                          
                                                                                                  
 conv2d_1011 (Conv2D)        (None, 5, 5, 192)            399360    ['block8_9_ac[0][0]']         
                                                                                                  
 conv2d_1014 (Conv2D)        (None, 5, 5, 256)            172032    ['activation_1013[0][0]']     
                                                                                                  
 batch_nor

In [86]:
from tensorflow.keras.callbacks import TensorBoard

In [89]:
tfb= TensorBoard('object detection')


In [90]:
#model training
model.fit(x=x_train,y=y_train,batch_size=10,epochs=100,validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/100
18/18 [==============================] - 63s 4s/step - loss: 0.0177 - accuracy: 0.5667 - val_loss: 0.0161 - val_accuracy: 0.5778
Epoch 2/100
18/18 [==============================] - 44s 2s/step - loss: 0.0077 - accuracy: 0.7611 - val_loss: 0.0088 - val_accuracy: 0.7556
Epoch 3/100
18/18 [==============================] - 42s 2s/step - loss: 0.0045 - accuracy: 0.8222 - val_loss: 0.0103 - val_accuracy: 0.8000
Epoch 4/100
18/18 [==============================] - 42s 2s/step - loss: 0.0040 - accuracy: 0.8278 - val_loss: 0.0069 - val_accuracy: 0.8444
Epoch 5/100
18/18 [==============================] - 42s 2s/step - loss: 0.0038 - accuracy: 0.8556 - val_loss: 0.0077 - val_accuracy: 0.8667
Epoch 6/100
18/18 [==============================] - 43s 2s/step - loss: 0.0023 - accuracy: 0.8722 - val_loss: 0.0079 - val_accuracy: 0.8667
Epoch 7/100
18/18 [==============================] - 42s 2s/step - loss: 0.0020 - accuracy: 0.9389 - val_loss: 0.0068 - val_accuracy: 0.8667
Epoch 8/100
1

18/18 [==============================] - 51s 3s/step - loss: 4.6744e-04 - accuracy: 0.9889 - val_loss: 0.0063 - val_accuracy: 0.8667
Epoch 58/100
18/18 [==============================] - 56s 3s/step - loss: 3.8714e-04 - accuracy: 0.9611 - val_loss: 0.0062 - val_accuracy: 0.8667
Epoch 59/100
18/18 [==============================] - 49s 3s/step - loss: 3.6511e-04 - accuracy: 0.9889 - val_loss: 0.0063 - val_accuracy: 0.8667
Epoch 60/100
18/18 [==============================] - 55s 3s/step - loss: 3.3423e-04 - accuracy: 0.9722 - val_loss: 0.0061 - val_accuracy: 0.8667
Epoch 61/100
18/18 [==============================] - 51s 3s/step - loss: 3.4022e-04 - accuracy: 0.9611 - val_loss: 0.0061 - val_accuracy: 0.8667
Epoch 62/100
18/18 [==============================] - 59s 3s/step - loss: 3.1071e-04 - accuracy: 0.9778 - val_loss: 0.0065 - val_accuracy: 0.8667
Epoch 63/100
18/18 [==============================] - 57s 3s/step - loss: 2.4934e-04 - accuracy: 0.9944 - val_loss: 0.0063 - val_accuracy

In [91]:
model.save("./model/object_detection.h5")

In [92]:
y_p=model.predict(x_test)
y_p

2/2 [==============================] - 13s 2s/step


array([[0.28672972, 0.7258641 , 0.6141445 , 0.49387532],
       [0.34957513, 0.5555032 , 0.65522975, 0.53977275],
       [0.30823934, 0.55726516, 0.8016782 , 0.68529916],
       [0.31824493, 0.6146628 , 0.5364548 , 0.39897087],
       [0.34617493, 0.7139295 , 0.5508994 , 0.41954383],
       [0.26090002, 0.7962253 , 0.64941823, 0.49709263],
       [0.3976597 , 0.7243664 , 0.4893132 , 0.37176996],
       [0.28035414, 0.7042374 , 0.5054449 , 0.33250284],
       [0.19728053, 0.4491568 , 0.7373578 , 0.63546205],
       [0.23380738, 0.54886854, 0.5636096 , 0.48129582],
       [0.3992553 , 0.6244675 , 0.68185985, 0.60464215],
       [0.23071145, 0.4719157 , 0.6438774 , 0.6161802 ],
       [0.4188421 , 0.8226547 , 0.5553773 , 0.46317983],
       [0.25090045, 0.7789082 , 0.6549578 , 0.34135154],
       [0.43374068, 0.6847888 , 0.58806556, 0.5526008 ],
       [0.070174  , 0.35131392, 0.687762  , 0.6205957 ],
       [0.44803676, 0.68867135, 0.6115584 , 0.5289871 ],
       [0.26232633, 0.5786478 ,

In [93]:
from sklearn.metrics import accuracy_score

loss,accuracy=model.evaluate(x_test,y_test)
print("accuracy of the model is:",accuracy*100)
print("loss:",loss)

2/2 [==============================] - 6s 2s/step - loss: 0.0067 - accuracy: 0.8222
accuracy of the model is: 82.22222328186035
loss: 0.0066692722029984


In [94]:
model.evaluate(x_train,y_train)

6/6 [==============================] - 25s 4s/step - loss: 4.4341e-04 - accuracy: 0.9500


[0.0004434130387380719, 0.949999988079071]